# <center>Degree Progress Analytics</center>

<center>
    <b>Gregory L. Heileman,$^\star$ Hayden W. Free$^\ddagger$ and Chaouki T. Abdallah$^\bullet$</b> <br><br>
    $^\star$Department of Electrical & Computer Engineering <br>
    University of Arizona <br>
    heileman@arizona.edu <br><br>
    $^\ddagger$Department of Computer Science <br>
    University of Kentucky <br>
    hayden.free@uky.edu<br><br>
    $^\bullet$Department of Electrical & Computer Engineering <br>
    Georgia Institute of Technology <br>
    ctabdallah@gatech.edu
</center>

## Introduction
The Curricular Analytics Toolbox includes capablities that allow you to create customized degree plans according to various user-specifed optimization criteria. In order to use these capabilities, you first must have permission to access the CurricularOptimziation.jl package through a professional-level account. To obtain a professional-level account, please visit: http://CurricularAnalytics.org/pro. 

After obtaining your professional account, in order to use the CurricularOptimization tools, first enter package mode from the Julia REPL by pressing `]`, then enter: 

`pkg> add https://github.com/CurricularAnalytics/CurricularOptimization.jl.git`

The curricular optimization features make use of the [JuMP](https://github.com/JuliaOpt/JuMP.jl) domain-specific language for specifying optimization problems in Julia, and call the [Gurobi](https://www.gurobi.com) solver in order to solve the optimzaton problems. In order to use these features you must also install the solver, called [Gurobi Optimizer](https://www.gurobi.com/downloads/gurobi-optimizer-eula). Gurobi is a commercial product, and requires a license key; however, [academic licenses](https://www.gurobi.com/downloads/end-user-license-agreement-academic) are available at no cost.

After installing the Gurobi Solver on your computer, execute the following commands to use the optimization capabilities within the Curricular Analytics Toolbox: 

In [11]:
using CurricularAnalytics, CurricularVisualization, CurricularOptimization

# Introduction
The ability to determine a student's progress towards degree completion is a fundamental capability used throughout higher education.
Specifically, given a collection of degree requirements a student must satisfy in order to earn the credential associated with an academic program, we are interested in answering questions related to degree progress relative to a student record.  A student record is assumed to contain the set of courses a student has attempted to complete, along with the grades they have earned on these attempts.  We are interested in answering the following types of questions:

- **Degree Completion Progress**. For a given degree program at a particular institution, which degree requirements are satisfied by a given student transcript?  Note: degree completion progress capabilities can be used to perform "degree audits."
- **Cohort Analytics**. This involve the aggregation of individual degree progress analyses for the purpose of answering cohort-based analytics questions. E.g., what is the average progress of a particular first-time full-time cohort, and the expected graduation rate of the cohort based upon this progress?
- **Transfer Articulation**. Given a student transcript from institution $A$, and a transfer articulation plan that details how courses at institution $A$ transfer as course credits to institution $B$, determine the degree requirements satisfied by the student transcript at institution $B$?  This capability can be used to support what-if analyses over an entire system of schools. E.g., given a student record from institution $A$, and a collection of institutions $\Omega = (B, C, D, \ldots)$, along with the transfer articulation plans from $A$ to every institution in $\Omega$, for each program at each institution in $\Omega$, what degree requirements are satisfied by the student record through transfer articulation? Based on these results, a student may wish to know which of the degrees offered by the institutions in $\Omega$ they are closest to earning.
- **Reverse Transfer Articulation**. Given a student transcript that includes credits from institutions $A$ and $B$, and a a transfer articulation plan that details how courses at institution $B$ transfer as course credits to institution $A$, determine if the entire set of degree requirements for any program at institution $A$ can be satisfied by transferring some credits from institution $B$.

## Degree Requirements, Degree Plans, and Curricula
In order to better understand this work, it is helpful to establish some terminology related to academic degree programs. Any degree program (aka, major or academic program) at a college has a set of *degree requirements* that must be satisfied by students in order for them to earn the degree associated with the program.  For instance, a program at the undergraduate level may require its students to satisfy a general education core, major requirements, elective requirements, minor requirements, etc. For instance, in order for a student to earn the BA in Biology degree, they must complete a set of courses that satisfy all of the degree requirements associated with BA in Biology pogram.

The particular collection of courses a student uses to satisfy a program's degree requirements is referred to as a *curriculum*. It is generally the case that many different curricula can satisfy a given set of degree requirements. For instance, students are given freedom in how to select the courses needed to satisfy technical electives, general electives, etc., and each choice produces a slightly different curriculum. The notion that a single set of degree requirements can be satisfied by numerous curricula is depicted in Figure 1. 
In summary, a curriculum specifies a set of courses, that if completed while earning sufficient grades, will lead to the conferral of a degree.

<img src="data-types.pdf" width="600">
<center>Figure 1. The relationships between degree requirements, curricula and degree plans.</center>

It is worth noting that the courses in a curriculum often have prerequisite arrangements between them which create constraints on the order in which courses may be taken. More broadly, a degree plan for completing a curriculum over a certain number of terms is often provided to students. That is, a *degree plan* includes a temporal component that stipulates how a curriculum should be organized on a term-by-term basis. Figure 1 depicts the fact that, depending upon how courses are arranged, many different degree plans can be constructed from a single curriculum.

### Combinatorial Complexity
How many different curricula can be constructed that satisfy a single set of degree requriements, and how many degree plans can be constructed that satisfy a single curriculum?

### Representing Degree Requirements
A convenient way of representing a set of degree requirements is as a *requirements tree*, as shown in Figure 2.

<img src="requirements-ex.pdf" width="800">
<center>Figure 2. An example requirement tree associated with a degree program. Requirements are shown as rectangles, and the courses that must be successfully completed in order to satisfy a requirement are shown as circles.</center>

As a simple example, consider a basket weaving program consisting of the following set of degree requirements:

- A set of *general educaiton core* requirements, consisting of math and english sub-requirements. 
- A set of *major requirements* specific to the basket weaving major.


## Computing Degree Progress 
A brief overview of how we have structured the degree requirements satisfaction problem process as a mathematical optimzation problem is provided next. If you'd like to get straight to using the optimization features, you may skip to the code sections below.

The formal framework we will use to represent a set of degree requirements in a requirements tree involves two types of structures, one for storing a collection of course requirements, i.e., course sets, and the other for storing a collection of requirements, i.e., requirement sets, as shown in Figure 3.

<img src="requirement.pdf" width="600">
<center>The two types of structures used to construct a requirements tree. <strong>(a)</strong> A <em>course set</em> requirement consists of a collection of course/minimum grade pairs $\{\rho_{1j}, \ldots, \rho_{jk}\}$, as well as the number of credit hours taken from the courses in $\{\rho_{1j}, \ldots, \rho_{jk}\}$ that must be successfully completed~(i.e., earn at least the minimum grade) in order to satisfy the requirement. <strong>(b)</strong> A <em>requirement set</em> consists of a set of requirements, i.e., course sets or other requirements sets, along with a specification of how many of them must be satisfied in order to satisfy the requirement set as a whole.</center>

A *course set* is simply a set of course requirements, along with the minimum number of credit hours required to satisfy the course set requirement.  More specifically, a single course-set requirement, $cs_j$, consists of two elements:
$cs_j = (\rho_j, \theta_j)$
where $\rho_j$ is a list of the $|\rho_j|$ courses in the course set, along with the minimum grade that must be earned in each, and $\theta_j$ is the number of credit hours that must be completed (while earning the minimum grades) from the courses contained in the course set.  We will use $\mbox{cr}[c_i]$ to denote the number of credit hours associated with course $c_i$.  With reference to a given student transcript, we will use $\mbox{gr}[c_i]$ to denote the highest grade value for course $c_i$ on the transcript, assigning the value zero if course $c_i$ is not on the transcript.  

> <u>**Course Set:**</u> Math General Education<br>
> **Mininum # of Credits:** 6<br>
> **Double Count:** True<br>
> **Course Requirements:**
>> (MATH 180: College Algebra (3 credits), D)<br> 
>>  (MATH 190: Trig. and Analytical Geometry (4 credits), D)<br> 
>>  (STA 120: Statistics and Probability (4 credits), D)<br> 
>>  (MATH 180: Calculus I (4 credits), D)<br> 
>>  (MATH 181: Calculus II (4 credits), D) 
 
This course set corresponds to the Math General Education requirement shown in Figure 2. It consists of five courses from which a student must earn D or better grades while accumulating at least six credit hours. In addition, the courses from this list may also be used to satisfy other degree requirements, i.e., they may be "double counted."   

A *requirement set* is a collection of other degree requirements, which may be course sets or other requirement sets as child requirements in the requirements. In addition, a requirement set specifies how many of the child requirements must be satisfied in order to satisfy the requirement set as a whole, and how many credit hours must be earned in doing so.  An example requirement set, corresponding to the General Education Core requirement shown in Figure 2, could then be specified as:

> <u>**Requirement Set:**</u> General Education Core<br>
> **Mininum # of Credits:** 30<br>
> **Satisfy:** All<br>
> **Requirements:** 
>> Math General Education<br>
>> English General Education<br>
>> Humanities General Education<br>
>> Fine Arts General Education 

This requirement set contains four child requirements, all of which must be satisfied in order to satisfy the requirement set as a whole. A total of 30 credit hours must be completed from among the courses in the course sets that are descendants of the requirement set in the requirements tree. 

We now demonstrate, using the Theorem state below, that the expressive power of the requirement tree formulation is universal (functionally complete); that is, it can be used to represent any set of degree requirements expressible as Boolean functions. (Note: A functionally complete set of Boolean operators is capable of representing all Boolean functions $f:\{0,1\}^n \rightarrow \{0,1\}, n \ge 1$.) This means that any set of degree requirements specified using propositional logic can be represented in a requirements tree, where satisfaction of the propositional logical formula in this case corresponds to satisfaction of the corresponding degree requirements.

**Theorem:** Any set of degree requirements expressible as a Boolean formula over a collection of courses can be represented using the previously described formal model for degree requirements.

**Proof:** Consider a collection of courses, $c_1, \ldots, c_n$, that may be used to satisfy a set of degree requirements. Create a collection of Boolean literals, $v_1, \ldots, v_n$, where $v_i$ evaluates true if course $c_i$ was completed while earning the minimum grade necessary for it to count towards the satisfaction of a degree requirement. (If the same course appears in more than one degree requirement with different minimum required grades, simply treat this as two courses.) Using these literals, construct a disjunctive normal form (DNF) Boolean expression that evaluates true if and only if the set of earned course grades satisfy the degree requirements.  Because DNF expressions are functionally complete, we will demonstrate how to convert any DNF expression into an equivalent requirement tree. Consider a DNF expression consisting of a disjunction of $k$ conjunctive clauses, each containing a subset of the $n$ literals.  For each of the $k$ clauses create a separate course set, where each course set contains the courses corresponding to the literals in its corresponding clause, and set the minimum number of required credits in each course set to be the sum of the credits associated with the courses in the course set.  Note that a conjunctive clause will only evaluate true if all of the literals it contains evaluate true, and that the course set constructed from this clause will only be satisfied if all of the courses it contains are successfully completed.  Next, consider the disjunction $(p \lor q)$ involving clauses $p$ and $q$, and assume the course set construction just described was used to construct course sets from clauses $p$ and $q$.  Construct three requirement sets using the course sets for $p$ and $q$, and organize them as follows:

<img src="requirement_set_construct.pdf" width="300">

where requirement sets $a$ and $b$ are set to satisfy one of their two child requirements, and requirement set $c$ is set to satisfy both of its child requirements. In addition, set both of the $p$ course sets and both of the $q$ course sets to allow double counting.  To see why the requirement set construction above will only be satisfied when $(p \lor q)$ evaluates true, notice that satisfying one of two child requirements in a requirement set is equivalent to an exclusive disjunction, and satisfying two of two child requirements in a requirement set is equivalent to a conjunction. Then, using the fact that  
$$
   (p \lor q) = (p \oplus q) \oplus (p \land q)
$$
the result follows.  Finally, use the same construction as above to create the necessary requirement sets associated with all of the remaining disjunctions in the original DNF expression.  QED



## Optimal Assignment Algorithm
Our goal is to find an assignment of courses to degree requirements that maximizes some objective function, $f(c_i)$, 
the number of credit hours from a student transcript that are applicable to the requirements in a particular degree program.
If $\mbox{cr}[c_i]$ denotes the number of credit hours associated with course $i$, then we can express this as the following integer linear programming (ILP) problem:
\begin{gather*}
   \mbox{Maximize} \; \sum_{i=1}^n \sum_{j=1}^m f(c_i) \cdot x_{ij}, \\
   \mbox{subject to:} \; 1) \ x_{ij} \in \{0,1\}, \ i = 1, \ldots, n, \ \ j = 1, \ldots m, \\
   \mbox{ 2) course set constraints, } \\
   \mbox{ 3) requirement set constraints.}
\end{gather*}

We consider next how to stipulate the various course set and requirement set constraints.

### Course-set Constraints
Given a requirement tree, the first set of constraints associated with course set requirements is given by the following set of linear inequalities:
 \begin{equation}
   \sum_{i =1}^n \left(\mbox{cr}[c_i] \cdot x_{ij}\right) \; \leq \;\theta_j, \ \ j = 1, \dots, m.
   \label{CH_constraint_cs}
 \end{equation}
These $m$ constraints ensure that no more than the minimum number of credit hours are used to satisfy each course set requirement.

When considering the course set $cs_j$, we will denote the $k$-th element in list $\rho_j = (\alpha_j, \gamma_j)$ using
$$
 \rho_{kj} = (\alpha_{kj}, \gamma_{\alpha_{kj},j}), \ \ k=1, \ldots, |\rho_j|,
$$
where $\alpha_{kj}$ is the row index in assignment matrix $x_{ij}$ of the $k$-th course in the list, and $\gamma_{\alpha_{kj},j}$ is the minimum grade that must be earned if course $c_{\alpha_{kj}}$ is to be applied in partially satisfying $cs_j$.
We will use $\langle B\rangle$, where $B$ is a boolean formula, to specify an indicator variable.  Specifically, 
$$
  \langle B \rangle = \left\{
  \begin{array}{ll}
  1; & \mbox{if $B$ evaluates as true,} \\
  0; & \mbox{otherwise.}
  \end{array}\right.
$$
Using this notation, the second set of constraints associated with course set requirements is given by the following set of indicator constraints:\footnote{For indicator constraint, $\langle B\rangle \implies$ constraint $\Gamma$, if $B$ is true, constraint $\Gamma$ will be added to the ILP problem.} 
 \begin{equation}
  \langle i \not\in \alpha_j \rangle \implies x_{ij} = 0, \ \ i = 1, \ldots, n; \ \ j = 1, \ldots, m.
  \label{no_class_constraint}
 \end{equation}
  \begin{equation}
  \langle i \in \alpha_j \rangle \cdot \langle \mbox{gr}[c_i] < \gamma_{ij}\rangle \implies x_{ij} = 0, \ \ i = 1, \ldots, n; \ \ j = 1, \ldots, m.
  \label{min_grade_constraint}
 \end{equation}
These constraints preclude course $c_i$ from being assigned to requirement set $cs_j$ when either $c_i$ is not in course set $cs_j$ (Equation~(no class constraint), or the minimum grade earned in that course is not sufficient (minimum grade constraint).

Finally, we will create a set of constraints that ensure a single course can only be used to partially satisfy a single requirement set, excluding the cases where courses may double count towards the satisfaction of  requirement sets: 
\begin{equation}
  \sum_{j=1}^m\left(\langle i \in \alpha_j \rangle \cdot \langle \mbox{dc}[c_i]  = \mbox{false}\rangle \cdot x_{ij}\right) = 1, \ \ i = 1, \ldots, n; \ \ j = 1, \ldots, m.
 \label{double_count_constraint}
\end{equation}
Note that if course $c_i$ is in course set $cs_j$, and it cannot be double counted, then variable $x_{ij}$ will be included in the summation associated with the constraints in the double count constraint. 

### Requirement-set Constraints
Given a requirement tree $R$, with a total of $q$ requirement sets, consider the subtree rooted at requirement set $rs_k$, and let $\mbox{cs}[rs_k]$ denote the collection of course sets that appear as leaves in this subtree.  Let $\mbox{idx}[\mbox{cs}[{rs_k}]]$ denote the set of column indices in assignment matrix $x_{ij}$ associated with the course sets associated belonging to requirement set $rs_k$. The following $q$ constraints ensure that no more than the minimum number of credit hours are used to satisfy a requirement set:
\begin{equation}
  \sum_{i=1}^n \sum_{j \in \mbox{idx}[\mbox{cs}[{rs_k}]]} \mbox{cr}[c_i] \cdot x_{ij} \leq \mbox{cr}[rs_k], \ \ k=1, \ldots q.
  \label{CH_constraint_rs}
\end{equation}

In order to account for the choice options associated with requirements sets, we will create a collection of indicator variables for every requirement set in $R$ that contains choices. Specifically, if requirement set $rs_k$ involves choices, then $l = |\mbox{ch}[rs_k]|$ indicator variables, $y_{1k}, \ldots, y_{lk}$, will be created, where $\mbox{ch}[rs_k]$ denotes the $l$ children requirements associated with $rs_k$. In addition, if $rs_k$ requires the satisfaction of $p_k$ of the $l$ child requirements, and $p_k < l$, then the following constraints will be added to the optimization model:
\begin{equation}
  \sum_{t=1}^{|\mbox{ch}[rs_k]|} y_{tk} = p_k, \ \ k = 1, \ldots q.
  \label{choice_var_constraints}
\end{equation}
These indicator variables will be used to construct indicator constraints that will enforce the fact that only $p_k$ of the children requirements need to be satisfied in $rs_k$.  Specifically, 
\begin{equation}
 \langle y_{tk} = 0 \rangle \implies \sum_{i=1}^n x_{ij} = 0, \ \ t = 1, \ldots, |{\mbox{cs}[rs_k]}|; \ \ k=1, \ldots q; \ \ j \in \mbox{idx}[\mbox{cs}[{rs_k}]].  
 \label{choice_constraints}
\end{equation} 

### Criterion Functions
If our objective is to maximize the total number of earned credits applicable to the degree requirements, we let $f(c_i) = \mbox{cr}[c_i] $, where $\mbox{cr}[c_i]$ is the number of credit hours associated with course $c_i$. 
This function maximizes the number of credit hours from a student transcript that are applicable to the requirements in a particular degree program, and when used in the original formulation of the problem, it leads to an integer linear programming (ILP) formulation:
\begin{gather*}
   \mbox{Maximize} \; \sum_{i=1}^n \sum_{j=1}^m \mbox{cr}[c_i] \cdot x_{ij}, \\
\end{gather*}

Other possible criterion functions:  Maximize GPA, maximize GPA in major, others?



## Example - Northern Kentucky University Computer Science
In the following example we demonstrate a use case for both the degree plan completion and transfer articulation functionalities.

For the sake of conserving space most of the input to perform these steps is contained in the `nku-example.jl` file, which is included with this notebook. Snippets of the various actions are provided below.

In [12]:
include("./nku-example.jl")
display("text/html", "<style>.container { width:100% !important; }</style>")

```julia
# Create stores for institutions courses and catalog.
college = "KCTCS"
univ = "Northern Keuncky University"
KCTCS_catalog = CourseCatalog("2019-20 Academic Year", college)
NKU_catalog = CourseCatalog("2019-20 Academic Year", univ)
KCTCS_courses = Array{Course,1}()
NKU_courses = Array{Course,1}()

# Create the KCTCS Courses 
push!(KCTCS_courses, Course("Writing I", 3, prefix="ENG", num="101", institution=college))
...
# Add KCTCS courses to catalog
add_course!(KCTCS_catalog, KCTCS_courses)

# NKU Courses
push!(NKU_courses, Course("Great Archaeological Sites", 3, prefix="ANT", num="114", institution=univ))
...
# Add NKU courses to catalog
add_course!(NKU_catalog, NKU_courses)

# NKU General Education Requirement
# First Communication subrequirement
opt1 = CourseSet("Written Option 1", 3, [course(NKU_catalog, "ENG", "104", "Honors College Writing") => grade("D")])
...
# Construct full communication requirement set
comm = RequirementSet("Communication", 9, [written, oral], description="Gen. Ed. sub-requirement")

# Create CS Major Requirements
cs_dr = RequirementSet("Degree Requirements", 120, convert(Array{AbstractRequirement,1}, [gen_ed, cs_mr]), description="Degree Requirements for the BS Computer Science Degree")

# Create the transfer articulation map between KCTCS and NKU
NKU_xfer = TransferArticulation("NKU Transfer Articulation", univ, NKU_catalog)
add_transfer_catalog(NKU_xfer, KCTCS_catalog)
add_transfer_course(NKU_xfer, [course_id("ENG", "101", "College Writing", univ)], KCTCS_catalog.id, course_id("ENG", "101", "Writing I", college))
...

# Create a transcript involving KCTCS coursework
KCTCS_transcript = Array{CourseRecord,1}()
push!(KCTCS_transcript, CourseRecord(course(KCTCS_catalog, "CIT", "105", "Introduction to Computing"), grade("A")))
...

# Finally use the NKU_xfer object to find equivalent courses for all courses in KCTCS_transcript
```

In [13]:
# Determine the requirements satisfied at NKU by the articulated courses
model = assign_courses(NKU_equiv_transcript, cs_dr, [applied_credits, requirement_level])

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 28 rows, 78 columns and 463 nonzeros
Model fingerprint: 0xf7665fea
Variable types: 0 continuous, 78 integer (78 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 9e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+02]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives (1 combined) ...
---------------------------------------------------------------------------
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 (weighted) ...
---------------------------------------------------------------------------

Optimize a model with 28 rows, 78 columns and 463 nonzeros
Variable types: 0 continuous, 78 integer (78 binary)
Coefficient statistics:
  Matrix range     [1e+0

A JuMP Model
Maximization problem with:
Variables: 78
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 1 constraint
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 27 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 78 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Gurobi
Names registered in the model: x, y

Here's the assignment created by the optimization algorithm:

In [14]:
x = model.obj_dict[:x]
value.(x)

6×13 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [16]:
is_satisfied = satisfied(NKU_equiv_transcript, cs_dr, value.(x))
show_requirements(cs_dr, satisfied=is_satisfied)

LoadError: MethodError: no method matching satisfied(::Array{CourseRecord,1}, ::RequirementSet, ::Array{Float64,2})